In [2]:
import polars as pl
import altair as alt
import pandas as pd
import numpy as np
import sklearn

alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [3]:
df = pl.read_csv('data/train.csv')

In [4]:
df

label,text,id
str,str,str
"""Sport""","""21-Jähriger fällt wohl bis Sai…","""b5e99883a83f420793048898c8a411…"
"""Kultur""","""Erfundene Bilder zu Filmen, di…","""fc069d9fc54647ab94f56ccf01851c…"
"""Web""","""Der frischgekürte CEO Sundar P…","""91c992a510f94058bc4b417cbe8987…"
"""Wirtschaft""","""Putin: ""Einigung, dass wir Men…","""9b75bafdb2d640708182ded42b08e8…"
"""Inland""","""Estland sieht den künftigen ös…","""05cf5e5ccb3f4ba69987caa492a17a…"
…,…,…
"""Inland""","""Bernd Saurer war Bridge-Junior…","""6cf74432366c4490b17f368330e305…"
"""International""","""Sandhere soll in vergangener W…","""948a3e420d634287b33a1719af7b4b…"
"""Wirtschaft""","""Derzeit Konzeptgruppe in Berli…","""513ede275c3448a4a23820c765933c…"


In [5]:
base = alt.Chart(df).mark_bar().encode(
    x=alt.X('label:O', title='Text Topic', axis=alt.Axis(labelAngle=45)),
    y=alt.Y('count()', title='Frequency'),
    color=alt.Color('label:N', title=''),
    tooltip=[
        alt.Tooltip('label:O', title='Text Topic'),
        alt.Tooltip('count()', title='Frequency')
    ],
)

final_chart = (base).properties(
    width=800,
    height=400,
    title='Distribution of 9 Text Topics'
)

final_chart

alt.Chart(...)

In [7]:
import altair as alt
import pandas as pd
from models import get_tokenizer

# 1. Load tokenizer
tokenizer = get_tokenizer("deepset/gelectra-base")

# 2. Compute token count per example
def count_tokens(text):
    enc = tokenizer(
        str(text),
        add_special_tokens=True,
        truncation=False,
        padding=False,
        return_attention_mask=False,
    )
    return len(enc["input_ids"])

# assuming df already exists and has a column with the raw text
df = df.with_columns(
    pl.col("text").map_elements(count_tokens).alias("token_count")
)

Token indices sequence length is longer than the specified maximum sequence length for this model (827 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (806 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1341 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (806 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1341 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

In [8]:
token_dist_chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(
            "token_count:Q",
            bin=alt.Bin(maxbins=50),
            title="Number of tokens (deepset/gelectra-base)"
        ),
        y=alt.Y("count()", title="Number of examples"),
        tooltip=[
            alt.Tooltip("token_count:Q", bin=True, title="Token count"),
            alt.Tooltip("count()", title="Frequency")
        ],
    )
    .properties(
        width=800,
        height=400,
        title="Distribution of token lengths (deepset/gelectra-base)"
    )
)

token_dist_chart

alt.Chart(...)

In [11]:
import altair as alt
import pandas as pd
from models import get_tokenizer

# 1. Load tokenizer
tokenizer = get_tokenizer("uklfr/gottbert-base")

# 2. Compute token count per example
def count_tokens(text):
    enc = tokenizer(
        str(text),
        add_special_tokens=True,
        truncation=False,
        padding=False,
        return_attention_mask=False,
    )
    return len(enc["input_ids"])

# assuming df already exists and has a column with the raw text
df = df.with_columns(
    pl.col("text").map_elements(count_tokens).alias("token_count")
)

In [12]:
token_dist_chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(
            "token_count:Q",
            bin=alt.Bin(maxbins=50),
            title="Number of tokens (uklfr/gottbert-base)"
        ),
        y=alt.Y("count()", title="Number of examples"),
        tooltip=[
            alt.Tooltip("token_count:Q", bin=True, title="Token count"),
            alt.Tooltip("count()", title="Frequency")
        ],
    )
    .properties(
        width=800,
        height=400,
        title="Distribution of token lengths (uklfr/gottbert-base)"
    )
)

token_dist_chart

alt.Chart(...)

In [14]:
# Calculate IQR (Interquartile Range) for token counts
df_pd = df.to_pandas()
Q1 = df_pd['token_count'].quantile(0.25)
Q3 = df_pd['token_count'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Token count statistics:")
print(f"Q1 (25th percentile): {Q1}")
print(f"Q3 (75th percentile): {Q3}")
print(f"IQR: {IQR}")
print(f"Lower bound (Q1 - 1.5*IQR): {lower_bound}")
print(f"Upper bound (Q3 + 1.5*IQR): {upper_bound}")
print(f"\nTotal samples: {len(df_pd)}")
print(f"Samples outside IQR range: {((df_pd['token_count'] < lower_bound) | (df_pd['token_count'] > upper_bound)).sum()}")
print(f"Samples within IQR range: {((df_pd['token_count'] >= lower_bound) & (df_pd['token_count'] <= upper_bound)).sum()}")

Token count statistics:
Q1 (25th percentile): 258.0
Q3 (75th percentile): 676.0
IQR: 418.0
Lower bound (Q1 - 1.5*IQR): -369.0
Upper bound (Q3 + 1.5*IQR): 1303.0

Total samples: 9245
Samples outside IQR range: 308
Samples within IQR range: 8937


In [15]:
# Visualize with IQR bounds marked
import altair as alt

token_dist_with_iqr = (
    alt.Chart(df_pd)
    .mark_bar()
    .encode(
        x=alt.X(
            "token_count:Q",
            bin=alt.Bin(maxbins=50),
            title="Number of tokens (uklfr/gottbert-base)"
        ),
        y=alt.Y("count()", title="Number of examples"),
        tooltip=[
            alt.Tooltip("token_count:Q", bin=True, title="Token count"),
            alt.Tooltip("count()", title="Frequency")
        ],
    )
)

# Add vertical lines for IQR bounds
lower_line = alt.Chart(pd.DataFrame({'x': [lower_bound]})).mark_rule(color='red', strokeDash=[5, 5]).encode(x='x:Q')
upper_line = alt.Chart(pd.DataFrame({'x': [upper_bound]})).mark_rule(color='red', strokeDash=[5, 5]).encode(x='x:Q')

final_chart_with_iqr = (token_dist_with_iqr + lower_line + upper_line).properties(
    width=800,
    height=400,
    title="Token length distribution with IQR bounds (red dashed lines)"
)

final_chart_with_iqr

alt.LayerChart(...)

In [16]:
# Filter out outliers based on IQR
df_filtered = df.filter(
    (pl.col("token_count") >= lower_bound) & (pl.col("token_count") <= upper_bound)
)

print(f"Original dataset size: {len(df)}")
print(f"Filtered dataset size: {len(df_filtered)}")
print(f"Removed {len(df) - len(df_filtered)} outliers ({100 * (len(df) - len(df_filtered)) / len(df):.2f}%)")

# Save filtered dataset
df_filtered.write_csv('data/train_filtered.csv')
print("\nFiltered dataset saved to 'data/train_filtered.csv'")

Original dataset size: 9245
Filtered dataset size: 8937
Removed 308 outliers (3.33%)

Filtered dataset saved to 'data/train_filtered.csv'


In [18]:
token_dist_chart = (
    alt.Chart(df_filtered)
    .mark_bar()
    .encode(
        x=alt.X(
            "token_count:Q",
            bin=alt.Bin(maxbins=50),
            title="Number of tokens (uklfr/gottbert-base)"
        ),
        y=alt.Y("count()", title="Number of examples"),
        tooltip=[
            alt.Tooltip("token_count:Q", bin=True, title="Token count"),
            alt.Tooltip("count()", title="Frequency")
        ],
    )
    .properties(
        width=800,
        height=400,
        title="Distribution of token lengths (uklfr/gottbert-base)"
    )
)

token_dist_chart

alt.Chart(...)

In [19]:
df_test = pl.read_csv('data/test.csv')

In [20]:
import altair as alt
import pandas as pd
from models import get_tokenizer

# 1. Load tokenizer
tokenizer = get_tokenizer("uklfr/gottbert-base")

# 2. Compute token count per example
def count_tokens(text):
    enc = tokenizer(
        str(text),
        add_special_tokens=True,
        truncation=False,
        padding=False,
        return_attention_mask=False,
    )
    return len(enc["input_ids"])

# assuming df already exists and has a column with the raw text
df_test = df_test.with_columns(
    pl.col("text").map_elements(count_tokens).alias("token_count")
)

In [21]:
token_dist_chart = (
    alt.Chart(df_test)
    .mark_bar()
    .encode(
        x=alt.X(
            "token_count:Q",
            bin=alt.Bin(maxbins=50),
            title="Number of tokens (uklfr/gottbert-base)"
        ),
        y=alt.Y("count()", title="Number of examples"),
        tooltip=[
            alt.Tooltip("token_count:Q", bin=True, title="Token count"),
            alt.Tooltip("count()", title="Frequency")
        ],
    )
    .properties(
        width=800,
        height=400,
        title="Distribution of token lengths (uklfr/gottbert-base)"
    )
)

token_dist_chart

alt.Chart(...)

In [22]:
# Calculate IQR (Interquartile Range) for token counts
df_test_pd = df_test.to_pandas()
Q1 = df_test_pd['token_count'].quantile(0.25)
Q3 = df_test_pd['token_count'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Token count statistics:")
print(f"Q1 (25th percentile): {Q1}")
print(f"Q3 (75th percentile): {Q3}")
print(f"IQR: {IQR}")
print(f"Lower bound (Q1 - 1.5*IQR): {lower_bound}")
print(f"Upper bound (Q3 + 1.5*IQR): {upper_bound}")
print(f"\nTotal samples: {len(df_test_pd)}")
print(f"Samples outside IQR range: {((df_test_pd['token_count'] < lower_bound) | (df_test_pd['token_count'] > upper_bound)).sum()}")
print(f"Samples within IQR range: {((df_test_pd['token_count'] >= lower_bound) & (df_test_pd['token_count'] <= upper_bound)).sum()}")

Token count statistics:
Q1 (25th percentile): 254.0
Q3 (75th percentile): 673.25
IQR: 419.25
Lower bound (Q1 - 1.5*IQR): -374.875
Upper bound (Q3 + 1.5*IQR): 1302.125

Total samples: 1028
Samples outside IQR range: 37
Samples within IQR range: 991


In [23]:
# Visualize with IQR bounds marked
import altair as alt

token_dist_with_iqr = (
    alt.Chart(df_test_pd)
    .mark_bar()
    .encode(
        x=alt.X(
            "token_count:Q",
            bin=alt.Bin(maxbins=50),
            title="Number of tokens (uklfr/gottbert-base)"
        ),
        y=alt.Y("count()", title="Number of examples"),
        tooltip=[
            alt.Tooltip("token_count:Q", bin=True, title="Token count"),
            alt.Tooltip("count()", title="Frequency")
        ],
    )
)

# Add vertical lines for IQR bounds
lower_line = alt.Chart(pd.DataFrame({'x': [lower_bound]})).mark_rule(color='red', strokeDash=[5, 5]).encode(x='x:Q')
upper_line = alt.Chart(pd.DataFrame({'x': [upper_bound]})).mark_rule(color='red', strokeDash=[5, 5]).encode(x='x:Q')

final_chart_with_iqr = (token_dist_with_iqr + lower_line + upper_line).properties(
    width=800,
    height=400,
    title="Token length distribution with IQR bounds (red dashed lines)"
)

final_chart_with_iqr

alt.LayerChart(...)